In [44]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os

In [45]:
x_train = np.load('data/x_train.npy')
y_train = np.load('data/y_train.npy')
x_test = np.load('data/x_test.npy')
y_test = np.load('data/y_test.npy')

In [46]:
print(x_train.shape)
print(x_test.shape)

(5344, 224, 224, 3)
(654, 224, 224, 3)


In [47]:
print(y_train.shape)
print(y_test.shape)

(5344,)
(654,)


In [ ]:
#x_train /= 255????

In [48]:
from __future__ import print_function

import numpy as np
from random import shuffle
import time
import csv
from PIL import Image
import os
import tensorflow as tf
import keras
from keras.callbacks import EarlyStopping, LearningRateScheduler
from keras import initializers
from keras.optimizers import SGD
from keras.preprocessing import sequence
from keras.utils import np_utils
from keras.models import Sequential,load_model,Model
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import *
from keras.callbacks import CSVLogger
from keras import callbacks
from keras.preprocessing.image import ImageDataGenerator

from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
import sklearn.metrics as sklm
import lossprettifier
from VGG import *

In [49]:
# for reproducibility
np.random.seed(3768)

# use this environment flag to change which GPU to use 
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="0"  # specify which GPU(s) to be used

#Get TensorFlow session
def get_session(): 
  config = tf.ConfigProto() 
  config.gpu_options.allow_growth = True 
  return tf.Session(config=config) 
  
# One hot encoding of labels 
def dense_to_one_hot(labels_dense,num_clases=4):
  return np.eye(num_clases)[labels_dense]

In [50]:
# Preparing training and test sets
x_train, x_valid, y_train, y_valid = train_test_split(x_train, y_train, test_size=0.10, random_state=42)

In [51]:
y_train = dense_to_one_hot(y_train,num_clases=4)
y_valid= dense_to_one_hot(y_valid,num_clases=4)

In [52]:
#Image data generation for the training 
datagen = ImageDataGenerator(
               featurewise_center = False, 
               samplewise_center = False,  # set each sample mean to 0
               featurewise_std_normalization = True,  
               samplewise_std_normalization = False)  

datagen.fit(x_train) 
for i in range(len(x_test)):
      x_test[i] = datagen.standardize(x_test[i])

/home/reza/.local/lib/python3.6/site-packages/keras_preprocessing/image/image_data_generator.py:348: UserWarning: This ImageDataGenerator specifies `featurewise_std_normalization`, which overrides setting of `featurewise_center`.
  warnings.warn('This ImageDataGenerator specifies '


In [58]:
#Defining hyperparameters
batch_Size = 32
steps_Per_Epoch = 32
numEpochs = 50

#Instantating VGG19 model
model = VGG19((224,224,3),4) #VGG19_dense for revised VGG19, VGG19 for VGG19. Please pay attention to VGG16(), chnage the input shape and class number in VGG.py.

#Creating an optimizers
adaDelta = keras.optimizers.Adadelta(lr=1.0, rho=0.95)
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.95, nesterov=True)
model.compile(optimizer = sgd , loss = 'categorical_crossentropy', metrics = ['accuracy'])

#Creating early stopping 
earlystop = EarlyStopping(monitor = 'val_accuracy', min_delta = 0, patience = 50, verbose = 1, mode = 'auto', restore_best_weights = True)       

train_generator = datagen.flow(x_train, y_train, batch_size = batch_Size)
validation_generator = datagen.flow(x_valid, y_valid, batch_size = batch_Size)

# Model training
history = model.fit_generator(
    train_generator,
    steps_per_epoch = steps_Per_Epoch,
    validation_data = validation_generator, 
    validation_steps = 16,
    epochs = numEpochs,
    shuffle = True, 
    verbose = 1)

Epoch 1/50
32/32 [==============================] - 20s 627ms/step - loss: 1.1260 - accuracy: 0.4854 - val_loss: 0.8739 - val_accuracy: 0.5586
Epoch 2/50
32/32 [==============================] - 19s 602ms/step - loss: 0.9745 - accuracy: 0.5459 - val_loss: 0.7980 - val_accuracy: 0.6620
Epoch 3/50
32/32 [==============================] - 19s 591ms/step - loss: 0.8714 - accuracy: 0.6054 - val_loss: 0.6983 - val_accuracy: 0.6700
Epoch 4/50
32/32 [==============================] - 20s 609ms/step - loss: 0.7564 - accuracy: 0.6807 - val_loss: 0.4598 - val_accuracy: 0.7654
Epoch 5/50
32/32 [==============================] - 19s 604ms/step - loss: 0.7233 - accuracy: 0.7168 - val_loss: 0.9107 - val_accuracy: 0.7256
Epoch 6/50
32/32 [==============================] - 19s 592ms/step - loss: 0.6709 - accuracy: 0.7083 - val_loss: 0.6225 - val_accuracy: 0.7316
Epoch 7/50
32/32 [==============================] - 19s 593ms/step - loss: 0.6825 - accuracy: 0.7139 - val_loss: 0.4330 - val_accuracy: 0.7654

In [59]:
modelPath = "VGG19_COVID19.h5"
resultPath = 'VGG19_COVID19.txt'

In [60]:
#y_test_oh = dense_to_one_hot(y_test, num_clases=4)

# visualizing losses and accuracy
train_loss = history.history['loss']
val_loss = history.history['val_loss']

#Observing the losses but can be commented out as it's not mandatory 
reporter = lossprettifier.LossPrettifier(show_percentage=True)

for i in range(numEpochs-1):
    reporter(epoch=i, LossA = train_loss[i], LossAB = val_loss[i])

# Model evaluation 
score, acc = model.evaluate(x_test, y_test_oh, batch_size=batch_Size)
print("Accuracy:", acc)

#if acc>0.675:
model.save_weights(modelPath)

Epoch     0 | LossA: 1.13(+0.00%) 	| LossAB: 0.87(+0.00%) 	
Epoch     1 | LossA: 0.97(-13.46%) ▼	| LossAB: 0.80(-8.68%) ▼	
Epoch     2 | LossA: 0.88(-9.63%) ▼	| LossAB: 0.70(-12.50%) ▼	
Epoch     3 | LossA: 0.76(-14.12%) ▼	| LossAB: 0.46(-34.15%) ▼	
Epoch     4 | LossA: 0.72(-4.37%) ▼	| LossAB: 0.91(+98.05%) ▲	
Epoch     5 | LossA: 0.68(-5.74%) ▼	| LossAB: 0.62(-31.65%) ▼	
Epoch     6 | LossA: 0.68(+0.10%) ▲	| LossAB: 0.43(-30.45%) ▼	
Epoch     7 | LossA: 0.73(+7.55%) ▲	| LossAB: 0.60(+39.55%) ▲	
Epoch     8 | LossA: 0.71(-2.96%) ▼	| LossAB: 0.31(-48.18%) ▼	
Epoch     9 | LossA: 0.66(-6.91%) ▼	| LossAB: 0.75(+139.94%) ▲	
Epoch    10 | LossA: 0.68(+2.35%) ▲	| LossAB: 0.58(-23.05%) ▼	
Epoch    11 | LossA: 0.68(-0.49%) ▼	| LossAB: 0.69(+20.11%) ▲	
Epoch    12 | LossA: 0.64(-5.25%) ▼	| LossAB: 0.50(-28.67%) ▼	
Epoch    13 | LossA: 0.63(-2.21%) ▼	| LossAB: 0.42(-14.85%) ▼	
Epoch    14 | LossA: 0.60(-4.87%) ▼	| LossAB: 0.58(+36.84%) ▲	
Epoch    15 | LossA: 0.59(-1.51%) ▼	| LossAB: 0.60(+3.61

In [61]:
y_pred = model.predict(x_test)
y_pred = y_pred.reshape(len(y_test), 4)
y_pred = np.argmax(y_pred, axis=1)

# Writing results on file
f = open(resultPath,'a') #create classification report
f.write(classification_report(y_test, y_pred))
f.write(str(sklm.cohen_kappa_score(y_test, y_pred))+","+str(acc)+","+str(score)+"\n")

#Print class-wise classification metrics
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.91      0.35      0.51       234
           1       0.62      0.96      0.75       246
           2       0.74      0.73      0.73       149
           3       0.36      0.56      0.44        25

    accuracy                           0.67       654
   macro avg       0.66      0.65      0.61       654
weighted avg       0.74      0.67      0.65       654

